## This notebook reads OSM data and creates a grid of cells tagged by their primary land use or road type label.

In [1]:
import numpy as np
import csv
import pandas as pd

from shapely.geometry import Point, MultiPolygon, Polygon#, mapping
import shapely
import folium
import json

from mapping import Mapping

from constants import Constants
from osm_reader import OSMReader
from grid_definition import GridDefinition

grid_definition = GridDefinition()
grid_definition.init()
constants = Constants()
GRID_SIZE = constants.getGridSize()
print(GRID_SIZE)
mapping = Mapping()

osm_reader = OSMReader()
osm_reader.init()
constants = Constants()
print(folium.__version__)

#Note: command to make the appropriate files
# ogr2ogr -f GeoJSON LeonMap-lines.geojson LeonMap.osm lines

60
0.5.0


## Retrieve road and land use classes and map them as lines and polygons, respectively

In [6]:
mapCenter = [grid_definition.getCenterLat(), grid_definition.getCenterLon()]
sum = 0

lineFilePath = '../osm/SmallLeon-lines.geojson'
roadGeos = osm_reader.getRoadGeoClasses(lineFilePath)
roadType = folium.Map(location=mapCenter, zoom_start=14,tiles="Stamen Toner")
for num, roadGeo in enumerate(roadGeos):
    sum += len(roadGeo["features"])
    osm_reader.addRoadType(roadType, roadGeo, num)
roadType.save("../images/maps/LeonSmallRoadType.html")
print(sum)

1415
1361


In [7]:
mapCenter = [grid_definition.getCenterLat(), grid_definition.getCenterLon()]

multiPolygonFilePath = '../osm/SmallLeon-multipolygons.geojson'
areaGeos = osm_reader.getLandGeoClasses(multiPolygonFilePath)

sum = 0

#print(areaGeos)
landUse = folium.Map(location=mapCenter, zoom_start=14, tiles="Stamen Toner")
for num, areaGeo in enumerate(areaGeos):
    #print(size(areaGeo))
    sum += len(areaGeo["features"])
    osm_reader.addLandUse(landUse, areaGeo, num)        
landUse.save("../images/maps/LeonSmallLandUse.html")
print(sum)

532
{'osm_id': '9093313', 'type': 'multipolygon', 'building': 'stadium'}
{'osm_way_id': '31308870', 'name': 'Panteon San Nicolas', 'landuse': 'cemetery'}
{'osm_way_id': '32403719', 'name': 'Alberca De la Salle', 'leisure': 'water_park', 'other_tags': '"Colonia"=>"Andrade"'}
{'osm_way_id': '202183624', 'leisure': 'track'}
{'osm_way_id': '230286916', 'name': 'Club Atenas', 'leisure': 'sports_centre'}
{'osm_way_id': '347709340', 'amenity': 'parking'}
{'osm_way_id': '347709341', 'amenity': 'parking'}
{'osm_way_id': '376152557', 'name': 'Auditorio Mateo Herrera', 'amenity': 'theatre', 'building': 'yes'}
{'osm_way_id': '376152558', 'name': 'Museo de Artes e Historia', 'building': 'yes', 'tourism': 'museum'}
{'osm_way_id': '376152562', 'name': 'Teatro del Bicentenario', 'amenity': 'theatre', 'building': 'yes'}
{'osm_way_id': '376153590', 'amenity': 'parking'}
{'osm_way_id': '376153593', 'name': 'La Estancia Argentina', 'amenity': 'restaurant', 'building': 'yes', 'other_tags': '"cuisine"=>"arg

In [ ]:
mapCenter = [grid_definition.getCenterLat(), grid_definition.getCenterLon()]

multiPolygonFilePath = '../osm/LeonMap-multipolygons.geojson'
with open(multiPolygonFilePath) as f:
            shapes = json.load(f)
features = shapes["features"]

areas1 = []

for feature in shapes["features"]:
    if "building" in feature["properties"]:
        if (feature["properties"]["building"] == "yes" and
            len(feature["properties"]) == 2):
            areas1.append(feature)

allLandUse = folium.Map(location=mapCenter, zoom_start=12, tiles="Stamen Toner")


#for feature in shapes["features"]:
areas1Geo = {
            'type': 'FeatureCollection',
            'features': areas1
        }
print(len(areas1Geo["features"]))
osm_reader.addLandUse(allLandUse, areas1, 4)  
allLandUse.save("../images/maps/LeonYesLandUse.html")

In [ ]:
landUse

In [ ]:
corners = constants.getCorners()

topLat = corners[0][0]
bottomLat = corners[2][0]

leftLon = corners[0][1]
rightLon = corners[1][1]

height = topLat - bottomLat
width = rightLon - leftLon
heightInterval = height / GRID_SIZE
widthInterval = width / GRID_SIZE

In [8]:
mapCenter = [grid_definition.getCenterLat(), grid_definition.getCenterLon()]

cells = osm_reader.getGeoCells()

## Map road type from csv

In [14]:
osm_dir = "/Users/zoepetard/Documents/Speckled/leon/osm/"
filename = osm_dir + "roadtype_grid" + str(GRID_SIZE) + ".csv"

#"0=Motorway, 1=Primary Road, 2=Secondary Road, 3=Tertiary or 'Unclassified' Road, 4=Residential or Service Road, 5=Pedestrian or Cycle Way, 6=Unknown/No Roads"
labels = ["Motorway", "Primary Road", "Secondary Road",
                   "Tertiary or 'Unclassified' Road", "Residential or Service Road", 
                   "Pedestrian or Cycle Way", "Unknown/No Roads"]

featureGroups = []
for label in labels:
    featureGroups.append(folium.FeatureGroup(name=label))

roadPixels = folium.Map(location=mapCenter,tiles=None, zoom_start=13, control_scale=True)
folium.TileLayer('Stamen Toner', name='Leon Road Types').add_to(roadPixels)

pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=1)
road_types = pd_df.values

for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    level = int(road_types[row][col])
    #print(level)
    osm_reader.addCellRoadType(featureGroups[level], cell_multi, level)

for featureGroup in featureGroups:
    #featureGroup = folium.FeatureGroup(name=labels[num])
    roadPixels.add_child(featureGroup)

sensor_feature_group = folium.FeatureGroup(name='Existing Sensors')
mapping.mapExistingSensors(sensor_feature_group, 'black')

roadPixels.add_child(sensor_feature_group)
    
roadPixels.add_child(folium.map.LayerControl())
roadPixels.save("../images/maps/LeonRoadTypeSmallGrid.html")

In [ ]:
##Back up
osm_dir = "/Users/zoepetard/Documents/Speckled/leon/osm/"
filename = osm_dir + "roadtype_grid" + str(GRID_SIZE) + ".csv"

#"0=Motorway, 1=Primary Road, 2=Secondary Road, 3=Tertiary or 'Unclassified' Road, 4=Residential or Service Road, 5=Pedestrian or Cycle Way, 6=Unknown/No Roads"
labels = np.array(["Motorway", "Primary Road", "Secondary Road",
                   "Tertiary or 'Unclassified' Road", "Residential or Service Road", 
                   "Pedestrian or Cycle Way", "Unknown/No Roads"])

featureGroups = np.empty(7, dtype=folium.FeatureGroup)

for num, featureGroup in enumerate(featureGroups):
    featureGroup = folium.FeatureGroup(name=labels[num])

roadPixels = folium.Map(location=mapCenter,tiles="Stamen Toner", zoom_start=14)
pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=1)
road_types = pd_df.values

for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    osm_reader.addCellRoadType(roadPixels, cell_multi, int(road_types[row][col]))

roadPixels.save("../images/maps/LeonCentroCellRoadType60WBlanks.html")
#roadPixels

## Map Land Use from CSV

In [11]:
osm_dir = "/Users/zoepetard/Documents/Speckled/leon/osm/"
filename = osm_dir + "landuse_grid" + str(GRID_SIZE) + ".csv"
        
landPixels = folium.Map(location=mapCenter,tiles=None, zoom_start=13, control_scale=True)
folium.TileLayer('Stamen Toner', name='Leon Land Uses').add_to(landPixels)

labels = ["Industrial", "Commercial", "Other Buildings",
                   "Residential", "Parks", 
                   "Water", "Unknown"]

featureGroups = []
for label in labels:
    featureGroups.append(folium.FeatureGroup(name=label))

pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=1)
land_use = pd_df.values

for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    index = int(land_use[row][col])
    osm_reader.addLandUse(featureGroups[index], cell_multi, index)
    
for featureGroup in featureGroups:
    #featureGroup = folium.FeatureGroup(name=labels[num])
    landPixels.add_child(featureGroup)
    
sensor_feature_group = folium.FeatureGroup(name='Existing Sensors')
mapping.mapExistingSensors(sensor_feature_group, 'black')
landPixels.add_child(sensor_feature_group)
    
landPixels.add_child(folium.map.LayerControl())

landPixels.save("../images/maps/ToShare/LeonLandUseSmallGrid.html")
#landPixels

## Make file with just green land use

In [ ]:
osm_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/osm/"
filename = osm_dir + "landuse_grid" + str(GRID_SIZE) + ".csv"

pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=1)

land_use = pd_df.values

cells = osm_reader.getGeoCells()


for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    index = int(land_use[row][col])
    if (index ==3):
        land_use[row][col] = 1
    else:
        land_use[row][col] = 0
        
osm_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/osm/"
filename = osm_dir + "landuse_green_grid" + str(GRID_SIZE) + ".csv"
with open(filename, 'w') as csvfile:
    csvfile.write("0=not green, 1=green \n")
    writer = csv.writer(csvfile, lineterminator='\n')
    writer.writerows(land_use)        


## Label cell geometry objects with major land use

In [9]:
## Only needs to be done once per test area
for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    
    LU_areas = []
    major_LU = 0
    major_LU_index = -1

    for LU_level, areaGeo in enumerate(areaGeos):
        features = areaGeo["features"]
        #print(len(features))
        area = 0
        for feat in range(len(features)):
            landuse_multi = areaGeo["features"][feat]["geometry"] 
            landuse_shape = shapely.geometry.asShape(landuse_multi)
            intersection = cell_shape.intersection(landuse_shape)
            area += intersection.area
        LU_areas.append(area)
        if (area > major_LU):
            major_LU = area
            major_LU_index = LU_level
    if (major_LU_index == -1): #if there are no landuses in the cell, make it grey
        major_LU_index = 6
    #osm_reader.addLandUse(pixelMap, cell_multi, major_LU_index)
    cells[cell_num][0]["properties"]["major_LU"] = major_LU_index
    
#iterate over first 0/1 to go through diff landuse collections (levels 1 to 6)
#iterate over second 0/1 to go through diff landuse items (items within a level)


## Write cell major land use grid to csv for future retrieval

In [10]:
## Only needs to be done once per test area
#cells = osm_reader.getCellsWithMajorLU()
cells_majorLU = np.zeros((GRID_SIZE, GRID_SIZE))

for cell_num, cell in enumerate(cells):
    #cell_multi = cell[0]["geometry"]
    major_LU = cell[0]["properties"]["major_LU"] #= major_LU_index
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    cells_majorLU[row][col] = major_LU

osm_dir = "/Users/zoepetard/Documents/Speckled/leon/osm/"
filename = osm_dir + "landuse_grid" + str(GRID_SIZE) + ".csv"
with open(filename, 'w') as csvfile:
    csvfile.write("1=industrial, 2=commercial, 3=buildings, 4=residential, 5=parks, 6=water, 7=unknown" + "\n")
    writer = csv.writer(csvfile, lineterminator='\n')
    writer.writerows(cells_majorLU)

## Label cell geometry objects with major road type.

In [12]:
## Only needs to be done once per test area
for cell_num, cell in enumerate(cells):
    cell_multi = cell[0]["geometry"]
    cell_shape = shapely.geometry.asShape(cell_multi)
    
    for road_level, roadGeo in enumerate(roadGeos):
        cells[cell_num][0]["properties"]["major_road"] = 6 #As a default, road level is grey/blank
        intersection = False 
        features = roadGeo["features"]
        for feat in range(len(features)):
            roadtype_multi = roadGeo["features"][feat]["geometry"] 
            roadtype_shape = shapely.geometry.asShape(roadtype_multi)
            intersection = cell_shape.intersects(roadtype_shape)
            if (intersection):
                cells[cell_num][0]["properties"]["major_road"] = road_level
                break
        if (intersection):
            break

## Write cell major road type grid to csv for future retrieval

In [13]:
## Only needs to be done once per test area
cells_major_road = np.zeros((GRID_SIZE, GRID_SIZE))

for cell_num, cell in enumerate(cells):
    major_road = cell[0]["properties"]["major_road"] 
    row = int(cell_num / GRID_SIZE)
    col = int(cell_num % GRID_SIZE)
    cells_major_road[row][col] = major_road

osm_dir = "/Users/zoepetard/Documents/Speckled/leon/osm/"
filename = osm_dir + "roadtype_grid" + str(GRID_SIZE) + ".csv"
with open(filename, 'w') as csvfile:
    csvfile.write("0=Motorway, 1=Primary Road, 2=Secondary Road, 3=Tertiary or 'Unclassified' Road, 4=Residential or Service Road, 5=Pedestrian or Cycle Way, 6=Unknown/No Roads" + "\n")
    writer = csv.writer(csvfile, lineterminator='\n')
    writer.writerows(cells_major_road)